### Links
- PyTorch MNIST example - https://github.com/pytorch/examples/tree/master/mnist
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
- LeNet for CIFAR-10 - https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py

My Google Doc - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
Trello - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
Using webcam - https://docs.google.com/document/d/17BVx5yioNaV11mip4U9gRAO7wLIJf-aOpIITJPjqeks/edit#
Installing pytorch - https://docs.google.com/document/d/1NfLbuhoOxonzS9FEeooyqpB90-62Gat-uZ2DOTaWRdg/edit#


In [1]:
# import sys
# sys.path.append("./kindling/")
# import kindling

In [2]:
import argparse

# from kindling.nan_police import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # in_channels, out_channels, kernel_size
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*4*4, 120)  # in_features, out_features
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        assert not torch.isnan(x).any().data, x
#         print(x.shape)
        out = F.relu(self.conv1(x))  # input
#         print("F.relu(self.conv1(x)):", out.shape)
        out = F.max_pool2d(out, 2)  # input, kernel_size
        assert not torch.isnan(out).any().data, out
#         print("F.max_pool2d(out, 2):", out.shape)
        out = F.relu(self.conv2(out))
        assert not torch.isnan(out).any().data, out
#         print("F.relu(self.conv2(out)):", out.shape)
        out = F.max_pool2d(out, 2)
        assert not torch.isnan(out).any().data, out
#         print("F.max_pool2d(out, 2):", out.shape)
        out = out.view(out.size(0), -1)  # collapse 
        assert not torch.isnan(out).any().data, out
#         print("out.view(out.size(0), -1):", out.shape)
        out = F.relu(self.fc1(out))
        assert not torch.isnan(out).any().data, out
#         print("F.relu(self.fc1(out)):", out.shape)
        out = F.relu(self.fc2(out))
        assert not torch.isnan(out).any().data, out
#         print("F.relu(self.fc2(out))", out.shape)
        out = self.fc3(out)
#         print("self.fc3(out)", out.shape)
        assert not torch.isnan(out).any().data, out
        return out


In [5]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.0001, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')


_StoreTrueAction(option_strings=['--save-model'], dest='save_model', nargs=0, const=True, default=False, type=None, choices=None, help='For Saving the current Model', metavar=None)

In [6]:
args = parser.parse_args("")
args

Namespace(batch_size=64, epochs=10, log_interval=10, lr=0.0001, momentum=0.5, no_cuda=False, save_model=False, seed=1, test_batch_size=1000)

In [7]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [8]:
torch.manual_seed(args.seed)

In [9]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [10]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [11]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)


In [12]:
model = Net().to(device)
model

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [13]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [14]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
#         print(data.shape, model)
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [20]:
for epoch in range(100000):
    train(args, model, device, train_loader, optimizer, epoch=epoch)

Train Epoch: 0 [0/60000 (0%)]	Loss: -0.051788
Train Epoch: 0 [640/60000 (1%)]	Loss: -0.036460
Train Epoch: 0 [1280/60000 (2%)]	Loss: -0.047059
Train Epoch: 0 [1920/60000 (3%)]	Loss: -0.044716
Train Epoch: 0 [2560/60000 (4%)]	Loss: -0.023716
Train Epoch: 0 [3200/60000 (5%)]	Loss: -0.035820
Train Epoch: 0 [3840/60000 (6%)]	Loss: -0.068132
Train Epoch: 0 [4480/60000 (7%)]	Loss: -0.038999
Train Epoch: 0 [5120/60000 (9%)]	Loss: -0.055866
Train Epoch: 0 [5760/60000 (10%)]	Loss: -0.045284
Train Epoch: 0 [6400/60000 (11%)]	Loss: -0.053773
Train Epoch: 0 [7040/60000 (12%)]	Loss: -0.048151
Train Epoch: 0 [7680/60000 (13%)]	Loss: -0.058392
Train Epoch: 0 [8320/60000 (14%)]	Loss: -0.047568
Train Epoch: 0 [8960/60000 (15%)]	Loss: -0.041861
Train Epoch: 0 [9600/60000 (16%)]	Loss: -0.067079
Train Epoch: 0 [10240/60000 (17%)]	Loss: -0.050545
Train Epoch: 0 [10880/60000 (18%)]	Loss: -0.062165
Train Epoch: 0 [11520/60000 (19%)]	Loss: -0.050176
Train Epoch: 0 [12160/60000 (20%)]	Loss: -0.047270
Train Epo

Train Epoch: 1 [43520/60000 (72%)]	Loss: -0.330977
Train Epoch: 1 [44160/60000 (74%)]	Loss: -0.348281
Train Epoch: 1 [44800/60000 (75%)]	Loss: -0.346607
Train Epoch: 1 [45440/60000 (76%)]	Loss: -0.396115
Train Epoch: 1 [46080/60000 (77%)]	Loss: -0.389788
Train Epoch: 1 [46720/60000 (78%)]	Loss: -0.341415
Train Epoch: 1 [47360/60000 (79%)]	Loss: -0.411320
Train Epoch: 1 [48000/60000 (80%)]	Loss: -0.423823
Train Epoch: 1 [48640/60000 (81%)]	Loss: -0.433130
Train Epoch: 1 [49280/60000 (82%)]	Loss: -0.462599
Train Epoch: 1 [49920/60000 (83%)]	Loss: -0.400152
Train Epoch: 1 [50560/60000 (84%)]	Loss: -0.469406
Train Epoch: 1 [51200/60000 (85%)]	Loss: -0.379051
Train Epoch: 1 [51840/60000 (86%)]	Loss: -0.427316
Train Epoch: 1 [52480/60000 (87%)]	Loss: -0.471935
Train Epoch: 1 [53120/60000 (88%)]	Loss: -0.428361
Train Epoch: 1 [53760/60000 (90%)]	Loss: -0.496678
Train Epoch: 1 [54400/60000 (91%)]	Loss: -0.474983
Train Epoch: 1 [55040/60000 (92%)]	Loss: -0.550163
Train Epoch: 1 [55680/60000 (93

AssertionError: tensor([[[[       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          ...,
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf]],

         [[       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          [       inf,        inf,        inf,  ...,        inf,
                  inf,        inf],
          ...,
          [       inf,        inf,        i

In [16]:
torch.optim.Optimizer

torch.optim.optimizer.Optimizer

In [17]:
type(optimizer)

torch.optim.sgd.SGD

In [18]:
isinstance(optimizer, torch.optim.SGD)

True

In [19]:
issubclass(type(optimizer), torch.optim.Optimizer)

True